In [6]:
# cd /home/nampv1/projects/vnpost_asr/voiceai.stt

In [11]:
# !pip install -r requirements.txt

In [1]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())

12.8
True


In [2]:
model_path = "/home/nampv1/models/vnpost-3"

In [7]:
pwd

'/home/nampv1/projects/vnpost_asr/voiceai.stt'

In [9]:
cd voiceai.stt

[Errno 2] No such file or directory: 'voiceai.stt'
/home/nampv1/projects/vnpost_asr/voiceai.stt


In [10]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor

/home/nampv1/anaconda3/envs/asr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
def get_model_path(model_name):
    """Get the model path of the model name."""
    middle = os.path.join(model_name, "snapshots")
    items = os.listdir(middle)
    subdirs = [d for d in items if os.path.isdir(os.path.join(middle, d))]

    if not subdirs:
        raise FileNotFoundError(f"No models found.")

    first_subdir = subdirs[0]
    model_path = os.path.join(middle, first_subdir)

    return model_path


In [12]:
import torch
import os

In [13]:
def load_model(model_name):
    """
    Load model and processor for the given model_name from its snapshot directory.
    This function returns (model, processor).
    """
    # Directory where ASR model snapshots are mounted inside Docker
    model_folder = "/home/nampv1/models/"
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Loading ASR model '{model_name}' on {device}...")

    model = None
    processor = None

    try:
        if model_name == "vnpost-1":
            base_model_dir = os.path.join(model_folder, "vnpost-2")
            print(f"  Locating snapshot for base model (vnpost-2)...")
            base_path = get_model_path(base_model_dir)

            base_processor = WhisperProcessor.from_pretrained(base_path)
            base_model = WhisperForConditionalGeneration.from_pretrained(base_path)

            adapter_dir = os.path.join(model_folder, "vnpost-1")
            model = PeftModel.from_pretrained(base_model, adapter_dir)
            processor = base_processor

            print(f"Loaded '{model_name}'")

        elif model_name == "vnpost-2":
            model_dir = os.path.join(model_folder, "vnpost-2")
            model_path = get_model_path(model_dir)

            processor = WhisperProcessor.from_pretrained(model_path)
            model = WhisperForConditionalGeneration.from_pretrained(model_path)
            print(f"Loaded model '{model_name}'")

        elif model_name == "vnpost-3":
            model_dir = os.path.join(model_folder, "vnpost-3")
            model_path = get_model_path(model_dir)

            processor = AutoProcessor.from_pretrained(model_path)
            model = AutoModelForSpeechSeq2Seq.from_pretrained(model_path)
            print(f"Loaded model '{model_name}'")

        else:
            raise ValueError(f"Undefined model name: {model_name}")

        # Move model to device and set to evaluation mode
        model.to(device)
        model.eval()
        print(f"Model '{model_name}' moved to {device} and set to eval mode.")

    except Exception as e:
        print(f"An unexpected error occurred while loading model '{model_name}': {e}")
        raise

    return model, processor

In [14]:
model_name = "vnpost-3"
model, processor = load_model(model_name)

Loading ASR model 'vnpost-3' on cuda...
Loaded model 'vnpost-3'
Model 'vnpost-3' moved to cuda and set to eval mode.


In [15]:
model

WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(128, 1280, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(1280, 1280, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 1280)
      (layers): ModuleList(
        (0-31): 32 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=1280, out_features=1280, bias=False)
            (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
            (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1280, out_features=5120, bias=True)
          (fc2): Linear(in_features=5120, out_features=1280, bias=Tr

In [21]:
model = model.to('cpu')

In [22]:
def transcribe_audio(audio_path_to_process, selected_models):
    """
    Transcribe the given audio file (original or enhanced) with each model in `selected_models`.
    Returns text results (all results), list for bar chart, and individual transcriptions
    for each model.
    """
    if not selected_models:
        return "Vui lòng chọn ít nhất một mô hình để so sánh.", None, None, None, []

    results = []
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Load audio using librosa (ensure 16kHz for Whisper)
    try:
        print(f"Loading audio for transcription from: {audio_path_to_process}")
        # Use librosa to load and ensure 16000 Hz sample rate
        audio_input_np, sr = librosa.load(audio_path_to_process, sr=16000)
        if sr != 16000:
            print(f"Warning: Audio sample rate is {sr}Hz. Resampling to 16000Hz for Whisper.")
        print(f"Audio loaded successfully, duration: {len(audio_input_np)/16000:.2f} seconds")
    except Exception as e:
        return f"Lỗi khi tải tệp âm thanh {os.path.basename(audio_path_to_process)}: {e}", None, None, None, []

    for model_name in selected_models:
        # Clear CUDA cache and garbage collect before loading/processing EACH model
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            reset_peak_memory_stats()  # Reset peak memory tracking
        gc.collect()
        print(f"\n--- Processing with Model: {model_name} ---")

        # Load model if not already loaded
        if model_name not in loaded_models:
            try:
                loaded_models[model_name], loaded_processors[model_name] = load_model(model_name)
            except Exception as e:
                print(f"Error loading model {model_name}: {e}")
                # Log the error and continue with the next model
                results.append({
                    "model": model_name,
                    "transcription": f"[Lỗi tải mô hình: {e}]",
                    "time_seconds": 0,
                    "memory_mb": 0
                })
                continue  # Skip this model

        model = loaded_models[model_name]
        processor = loaded_processors[model_name]

        # Measure initial GPU memory
        initial_gpu_memory = get_gpu_memory_usage()

        # Sync GPU before timing starts
        if torch.cuda.is_available():
            torch.cuda.synchronize()

        # Start measuring transcription time
        start_time = time.time()

        try:
            # Prepare input
            input_features = processor(audio_input_np, sampling_rate=16000, return_tensors="pt").input_features.to(device)

            gen_kwargs = {"language": "vi", "task": "transcribe"}

            print("Generating transcript...")
            with torch.no_grad():
                if hasattr(model, "generate"):
                    predicted_ids = model.generate(input_features, **gen_kwargs)
                else:
                    print(f"Warning: Model {model_name} may not have standard .generate method. Trying default.")
                    outputs = model(input_features)
                    predicted_ids = torch.argmax(outputs.logits, dim=-1)

            # Decode
            print("Decoding transcript...")
            if hasattr(processor, "batch_decode"):
                transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]
            elif hasattr(processor, "decode"):
                transcription = processor.decode(predicted_ids[0], skip_special_tokens=True)
            else:
                transcription = "[Lỗi giải mã: Bộ xử lý thiếu phương thức batch_decode hoặc decode]"
                print(f"Warning: Processor for {model_name} lacks standard decoding methods.")

            # Post-processing
            clean_transcription = spell_correct_vietnamese(
                transcription, 
                kenlm_model, 
                symspell_telex, 
                reverse_telex_map, 
                bert_model, 
                tokenizer,
                sym_max_telex_lookup_dist=2,  # Max distance in SymSpell lookup
                kenlm_low_score_threshold_per_word=-4.0  # Threshold for low kenlm score
            )

            # Sync GPU before ending timing
            if torch.cuda.is_available():
                torch.cuda.synchronize()

            end_time = time.time()
            time_taken = end_time - start_time

            peak_gpu_memory = get_peak_gpu_memory_usage()
            gpu_memory_used = peak_gpu_memory - initial_gpu_memory

            print(f"Transcript: {transcription}")
            print(f"Cleaned transcript: {clean_transcription}")
            print(f"Execution time: {time_taken:.2f} seconds")
            print(f"GPU memory used: {gpu_memory_used:.2f} MB")

            results.append({
                "model": model_name,
                "transcription": clean_transcription,
                "time_seconds": time_taken,
                "memory_mb": max(0, gpu_memory_used)
            })

        except Exception as e:
            print(f"Error during transcription with {model_name}: {e}")
            end_time = time.time()
            time_taken = end_time - start_time
            peak_gpu_memory = get_peak_gpu_memory_usage()
            gpu_memory_used = peak_gpu_memory - initial_gpu_memory
            results.append({
                "model": model_name,
                "transcription": f"[Lỗi chuyển đổi: {e}]",
                "time_seconds": time_taken,
                "memory_mb": max(0, gpu_memory_used)
            })
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
            gc.collect()

    # --- Format results for UI ---
    # Keep path in results for clarity
    text_results = f"Đã xử lý tệp: {os.path.basename(audio_path_to_process)}\n"
    text_results += "="*30 + "\n\n"
    for r in results:
        text_results += f"Mô hình: {r['model']}\n"
        text_results += f"Bảng ghi: {r['transcription']}\n"
        text_results += f"Thời gian: {r['time_seconds']:.2f} giây\n"
        text_results += f"Bộ nhớ GPU: {r['memory_mb']:.2f} MB\n"
        text_results += "-" * 30 + "\n"
    # --- End UI Result Formatting ---

    model_names = [r["model"] for r in results]
    times = [r["time_seconds"] for r in results]
    memories = [r["memory_mb"] for r in results]

    return text_results, model_names, times, memories, [r["transcription"] for r in results]


In [17]:
import librosa

In [19]:
from deploy.basic.main_nampv1 import *

In [26]:
audio_path = "/home/nampv1/projects/vnpost_asr/data/examples/audio.wav"

In [29]:
msg, model_names, time_vals, mem_vals, results = transcribe_audio(audio_path, ["vnpost-3"])

if results:
    for r in results:
        print(r["model"], r["transcription"], r["time_seconds"], r["memory_mb"])
else:
    print(msg)

Loading audio for transcription from: /home/nampv1/projects/vnpost_asr/data/examples/audio.wav
Audio loaded successfully, duration: 3.14 seconds

--- Processing with Model: vnpost-3 ---
Generating transcript...
Decoding transcript...

=== Processing Original Text: "không thờ ơ quá và cũng không quan tâm quá trong một thời điểm." ===
(Text normalized to: "không thờ ơ quá và cũng không quan tâm quá trong một thời điểm .")

--- Detecting Errors (Telex dict lookup for single words, KenLM score check) ---
thờ_ơ: -2.5176339149475098
quan_tâm: -2.185436964035034
thời_điểm: -2.1803297996520996
No errors detected when using pyvi postagging, switching to underthesea...

--- Detecting Errors (Telex dict lookup for single words, KenLM score check) ---
--- Error Detection Complete (0 errors found) ---

No potential errors detected based on KenLM scoring.
Transcript: không thờ ơ quá và cũng không quan tâm quá trong một thời điểm.
Cleaned transcript: không thờ ơ quá và cũng không quan tâm quá trong m

TypeError: string indices must be integers, not 'str'

In [33]:
print(msg)

Đã xử lý tệp: audio.wav

Mô hình: vnpost-3
Bảng ghi: không thờ ơ quá và cũng không quan tâm quá trong một thời điểm .
Thời gian: 1.02 giây
Bộ nhớ GPU: 92.44 MB
------------------------------



In [7]:
torch.__version__

'2.8.0+cu128'